In [ ]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 opencv-python mediapipe sklearn matplotlib

In [ ]:
!pip uninstall protobuf

In [9]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [10]:
#MediaPipe solutions - reconhecimento e desenho dos pontos na mão
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [11]:
def mediapipe_detection(image, model):
    """ 
    Funcao com objetivo de aplicar a previsao de reconhecimento na imagem desejada 
    Recebe imagem (frame da webcam) e um modelo que sera responsavel pela previsão
    Retorna o frame utilizado e a previsao feita pelo modelo
    A conversao da imagem se faz necessaria para ser tratada pelo modelo (BGR -> RGB)
    A mudanca na propriedade de leitura tem como objetivo salvar memoria
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [12]:
def draw_landmarks(image, results):
    """
    Funcao que recebe uma imagem (frame) junto com as previsoes feitas pelo modelo e aplica
    sob a imagem o desenho dos pontos necessarios (nao ha motivo para devolver a imagem pois ela
    ja e alterada diretamente).
    """
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [5]:
#Acessa a webcam - valor (0) representa o hardware
cap = cv2.VideoCapture(0)

#Define o modelo utilizado
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #Seleciona o frame atual da webcam
        ret, frame = cap.read()

        #Processa previsoes
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        #Desenha os pontos
        draw_landmarks(image, results)
        
        #Faz o display do frame
         
        cv2.imshow('Webcam', image)

        #Encerra a webcam 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [28]:
cap.release()
cv2.destroyAllWindows()

In [13]:
def extract_keypoints(results):
    '''É preciso guardar todos os pontos lidos pelo modelo em um numpy array
    Este array pose, por exemplo, armazena 33 numpy arrays que guardam, cada um, um ponto
    lido pelo modelo (com 3 coordenadas e visibility). Representando um único frame
    Depois o array é tratado de forma a transformá-lo num único array com todos os pontos.
    É retornado um array que junta todos os pontos lidos pela câmera.'''
    if results.pose_landmarks:      #Verifica se apareceu na câmera uma pessoa (array não vazio)
        pose = []
        for res in results.pose_landmarks.landmark:
            test = np.array([res.x, res.y, res.z, res.visibility])
            pose.append(test)

        np.array(pose).flatten()    #Trata os dados de forma a deixá-los todos em um único array de 33 vezes 4 = 132
    else:                           #Caso esteja vazio, cria um numpy array com zeros
        pose = np.zeros(132) 
        
    if results.left_hand_landmarks:  #Verifica se apareceu na câmera a mão esquerda (array não vazio)
        lefthand = []
        for res in results.left_hand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            lefthand.append(test)

        np.array(lefthand).flatten() #Trata os dados de forma a deixá-los todos em um único array de 21 vezes 3 = 63
    else:                            #Caso esteja vazio, cria um numpy array com zeros
        lefthand = np.zeros(63) 

    if results.right_hand_landmarks:  #Verifica se apareceu na câmera a mão direita (array não vazio)
        righthand = []
        for res in results.right_hand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            righthand.append(test)

        np.array(righthand).flatten() #Trata os dados de forma a deixá-los todos em um único array de 21 vezes 3 = 63
    else:                             #Caso esteja vazio, cria um numpy array com zeros
        righthand = np.zeros(63) 

    if results.face_landmarks:     #Verifica se apareceu na câmera a mão direita (array não vazio)
        face = []
        for res in results.face_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            face.append(test)

        np.array(face).flatten()    #Trata os dados de forma a deixá-los todos em um único array de 468 vezes 3 = 1404
    else:                           #Caso esteja vazio, cria um numpy array com zeros
        face = np.zeros(1404) 
    
    return np.concatenate([pose, face, lefthand, righthand]) #Concatenação de todos pontos

In [14]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [15]:
# Caminho que guarda o numpy array com os pontos extraidos
DATA_PATH = os.path.join('MP_Data')

# Sinais que serão detectados
actions = np.array(['ola', 'obrigado', 'teamo'])

# Representa a quantidade de sequências de frames que tem os dados
no_sequences = 30

# Representa a quantidade de frames que cada sequência possui
sequence_lenght = 30

In [10]:
# Cria pastas para cada sinal definido
# Cada pasta tem 30 pastas representando os videos modelo do sinal
# Cada vídeo contém 30 frames e cada frame 1662 pontos extraidos
for action in actions:
    for sequence in range(no_sequences):
        try: # Cria pastas e subpastas
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except: # Caso já exista passa para próxima pasta
            pass

In [22]:
#Acessa a webcam - valor (0) representa o hardware
cap = cv2.VideoCapture(0)

#Define o modelo utilizado
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Extrai os frames de cada video para cada ação
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_lenght):

                #Seleciona o frame atual da webcam
                ret, frame = cap.read()

                #Processa previsoes
                image, results = mediapipe_detection(frame, holistic)
                print(results)

                #Desenha os pontos
                draw_landmarks(image, results)
                
                # Interface e espera ao começar gravações
                if frame_num == 0:
                    
                    #Textos para indicar inicio de gravação
                    cv2.putText(image, "INICIANDO GRAVAÇÃO DE SINAL", 
                    (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, "Gravando para {} Video: {}".format(action, sequence), 
                    (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    # Tempo de espera entre um vídeo e outro
                    cv2.waitKey(1500)
                else:
                    cv2.putText(image, "Gravando para {} Video: {} Frame: {}".format(action, sequence, frame_num), 
                    (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                # Extrai os pontos de um frame e salva como um arquivo .npy (numpy array)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                #Faz o display do frame
                if ret == True: 
                    cv2.imshow('Webcam', image)

                    #Encerra a webcam 
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                else:
                    break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [9]:
# Imports que serão importante para treinar o modelo e nomear os dados
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []

# Carrega e adiciona todos os pontos de uma sequência no array sequences
# Enquanto o array labels coloca em ordem o valor de determinada ação com certa sequencia
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_lenght):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence),'{}.npy'.format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    
    matrix_labels = to_categorical(labels).astype(int)
    
    # Com os dados já definidos é possível separar uma quantidade para teste e treinamento do modelo
    x_train, x_test, y_train, y_test = train_test_split(np.array(sequences), matrix_labels, test_size=0.05)

In [11]:
# Imports para construção da rede neural
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [12]:
# Definição do caminho para registro de logs da rede
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [13]:
# Setup da rede neural
model = Sequential()

# Cada função add adiciona uma layer para a rede
# O parâmetro return_sequences é importante para passar para próxima layer de forma ordenada
# O parâmetro activation define a função que será utilizada para ativar os neurônios
#  O parâmetro input_shape = 30 frames cada um com 1662 pontos reconhecidos
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) 
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Última camada possui a quantidades de pontos definido pela quantidade de ações
# A função softmax entrega um resultado com a probabilidade de cada ação (será selecionado a com maior probabilidade)
model.add(Dense(actions.shape[0], activation='softmax'))

In [14]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [15]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 4s 113ms/step - loss: 1.1908 - categorical_accuracy: 0.3412
Epoch 2/2000
3/3 [==============================] - 0s 105ms/step - loss: 3.3637 - categorical_accuracy: 0.3294
Epoch 3/2000
3/3 [==============================] - 0s 110ms/step - loss: 1.1783 - categorical_accuracy: 0.3059
Epoch 4/2000
3/3 [==============================] - 0s 103ms/step - loss: 2.4209 - categorical_accuracy: 0.3765
Epoch 5/2000
3/3 [==============================] - 0s 107ms/step - loss: 1.1402 - categorical_accuracy: 0.3882
Epoch 6/2000
3/3 [==============================] - 0s 124ms/step - loss: 1.2578 - categorical_accuracy: 0.3529
Epoch 7/2000
3/3 [==============================] - 0s 112ms/step - loss: 3.0309 - categorical_accuracy: 0.3529
Epoch 8/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.9278 - categorical_accuracy: 0.3647
Epoch 9/2000
3/3 [==============================] - 0s 111ms/step - loss: 1.9117 - categorical_accuracy:

3/3 [==============================] - 0s 106ms/step - loss: 4.9455 - categorical_accuracy: 0.3294
Epoch 145/2000
3/3 [==============================] - 0s 110ms/step - loss: 4.3075 - categorical_accuracy: 0.3529
Epoch 146/2000
3/3 [==============================] - 0s 103ms/step - loss: 3.8123 - categorical_accuracy: 0.3529
Epoch 147/2000
3/3 [==============================] - 0s 110ms/step - loss: 3.9957 - categorical_accuracy: 0.3647
Epoch 148/2000
3/3 [==============================] - 0s 98ms/step - loss: 2.5184 - categorical_accuracy: 0.3176
Epoch 149/2000
3/3 [==============================] - 0s 128ms/step - loss: 2.9844 - categorical_accuracy: 0.3059
Epoch 150/2000
3/3 [==============================] - 0s 147ms/step - loss: 2.1831 - categorical_accuracy: 0.3176
Epoch 151/2000
3/3 [==============================] - 0s 151ms/step - loss: 2.1636 - categorical_accuracy: 0.3294
Epoch 152/2000
3/3 [==============================] - 0s 151ms/step - loss: 1.6325 - categorical_accurac

3/3 [==============================] - 0s 161ms/step - loss: 1.5566 - categorical_accuracy: 0.3647
Epoch 217/2000
3/3 [==============================] - 0s 105ms/step - loss: 1.5456 - categorical_accuracy: 0.3294
Epoch 218/2000
3/3 [==============================] - 0s 105ms/step - loss: 1.6019 - categorical_accuracy: 0.4353
Epoch 219/2000
3/3 [==============================] - 0s 152ms/step - loss: 2.0367 - categorical_accuracy: 0.3882
Epoch 220/2000
3/3 [==============================] - 0s 160ms/step - loss: 2.1310 - categorical_accuracy: 0.4353
Epoch 221/2000
3/3 [==============================] - 0s 101ms/step - loss: 1.3734 - categorical_accuracy: 0.4824
Epoch 222/2000
3/3 [==============================] - 0s 146ms/step - loss: 3.2012 - categorical_accuracy: 0.4353
Epoch 223/2000
3/3 [==============================] - 0s 151ms/step - loss: 1.9372 - categorical_accuracy: 0.3882
Epoch 224/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.5989 - categorical_accura

3/3 [==============================] - 0s 101ms/step - loss: 1.1511 - categorical_accuracy: 0.4235
Epoch 289/2000
3/3 [==============================] - 0s 143ms/step - loss: 0.8648 - categorical_accuracy: 0.5529
Epoch 290/2000
3/3 [==============================] - 0s 115ms/step - loss: 0.9931 - categorical_accuracy: 0.4706
Epoch 291/2000
3/3 [==============================] - 0s 102ms/step - loss: 0.9762 - categorical_accuracy: 0.6000
Epoch 292/2000
3/3 [==============================] - 0s 150ms/step - loss: 1.0386 - categorical_accuracy: 0.4235
Epoch 293/2000
3/3 [==============================] - 0s 108ms/step - loss: 0.9078 - categorical_accuracy: 0.5647
Epoch 294/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.7948 - categorical_accuracy: 0.6353
Epoch 295/2000
3/3 [==============================] - 0s 99ms/step - loss: 0.8962 - categorical_accuracy: 0.5529
Epoch 296/2000
3/3 [==============================] - 0s 114ms/step - loss: 0.8965 - categorical_accurac

3/3 [==============================] - 0s 104ms/step - loss: 0.7354 - categorical_accuracy: 0.6941
Epoch 361/2000
3/3 [==============================] - 0s 143ms/step - loss: 0.7252 - categorical_accuracy: 0.6824
Epoch 362/2000
3/3 [==============================] - 0s 115ms/step - loss: 0.7943 - categorical_accuracy: 0.5882
Epoch 363/2000
3/3 [==============================] - 0s 114ms/step - loss: 0.8346 - categorical_accuracy: 0.6000
Epoch 364/2000
3/3 [==============================] - 0s 96ms/step - loss: 0.8151 - categorical_accuracy: 0.6588
Epoch 365/2000
3/3 [==============================] - 0s 153ms/step - loss: 0.7979 - categorical_accuracy: 0.6353
Epoch 366/2000
3/3 [==============================] - 0s 110ms/step - loss: 0.8454 - categorical_accuracy: 0.5412
Epoch 367/2000
3/3 [==============================] - 0s 96ms/step - loss: 0.9492 - categorical_accuracy: 0.5647
Epoch 368/2000
3/3 [==============================] - 0s 165ms/step - loss: 0.8498 - categorical_accuracy

3/3 [==============================] - 0s 102ms/step - loss: 0.9328 - categorical_accuracy: 0.5647
Epoch 433/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.9557 - categorical_accuracy: 0.5765
Epoch 434/2000
3/3 [==============================] - 0s 103ms/step - loss: 0.8205 - categorical_accuracy: 0.6353
Epoch 435/2000
3/3 [==============================] - 0s 99ms/step - loss: 0.8631 - categorical_accuracy: 0.5882
Epoch 436/2000
3/3 [==============================] - 0s 103ms/step - loss: 0.8260 - categorical_accuracy: 0.6118
Epoch 437/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.7517 - categorical_accuracy: 0.6824
Epoch 438/2000
3/3 [==============================] - 0s 103ms/step - loss: 0.8132 - categorical_accuracy: 0.6353
Epoch 439/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.8625 - categorical_accuracy: 0.6353
Epoch 440/2000
3/3 [==============================] - 0s 99ms/step - loss: 0.8660 - categorical_accuracy: 0

3/3 [==============================] - 0s 115ms/step - loss: 0.3696 - categorical_accuracy: 0.8471
Epoch 577/2000
3/3 [==============================] - 0s 125ms/step - loss: 0.3823 - categorical_accuracy: 0.8824
Epoch 578/2000
3/3 [==============================] - 0s 141ms/step - loss: 0.4516 - categorical_accuracy: 0.8353
Epoch 579/2000
3/3 [==============================] - 0s 131ms/step - loss: 0.3587 - categorical_accuracy: 0.8706
Epoch 580/2000
3/3 [==============================] - 0s 115ms/step - loss: 0.3327 - categorical_accuracy: 0.8941
Epoch 581/2000
3/3 [==============================] - 0s 169ms/step - loss: 0.4220 - categorical_accuracy: 0.8706
Epoch 582/2000
3/3 [==============================] - 0s 141ms/step - loss: 0.3958 - categorical_accuracy: 0.8588
Epoch 583/2000
3/3 [==============================] - 0s 129ms/step - loss: 0.4604 - categorical_accuracy: 0.8588
Epoch 584/2000
3/3 [==============================] - 0s 152ms/step - loss: 0.4084 - categorical_accura

Epoch 648/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.2400 - categorical_accuracy: 0.9059
Epoch 649/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.2031 - categorical_accuracy: 0.9529
Epoch 650/2000
3/3 [==============================] - 0s 173ms/step - loss: 0.5434 - categorical_accuracy: 0.7529
Epoch 651/2000
3/3 [==============================] - 0s 150ms/step - loss: 0.5618 - categorical_accuracy: 0.8000
Epoch 652/2000
3/3 [==============================] - 0s 144ms/step - loss: 0.6697 - categorical_accuracy: 0.7412
Epoch 653/2000
3/3 [==============================] - 0s 105ms/step - loss: 0.2708 - categorical_accuracy: 0.8471
Epoch 654/2000
3/3 [==============================] - 0s 157ms/step - loss: 0.2692 - categorical_accuracy: 0.9059
Epoch 655/2000
3/3 [==============================] - 0s 149ms/step - loss: 0.2532 - categorical_accuracy: 0.9412
Epoch 656/2000
3/3 [==============================] - 0s 117ms/step - loss: 0.2291 - cate

KeyboardInterrupt: 

In [16]:
# Mostra a performance do modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [17]:
# Retorna uma lista com valores de probabilidade de cada ação como definidos no modelo
res = model.predict(x_test)

In [18]:
# Salva os weights usados pelo modelo
model.save('action.h5')
#model.load_weights(action.h5)

In [18]:
model.load_weights('action.h5')

In [19]:
# Teste de métricas para aprovação do modelo
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [20]:
yhat = model.predict(x_test)
#yhat = model.predict(x_train)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

# Retorna uma matriz bidimensional que representa se houve falsos positivos ou negativos
# Quanto maior a quantidade do valor de um lado da matriz melhor o modelo
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[4, 1],
        [0, 0]],

       [[2, 0],
        [1, 2]]], dtype=int64)

In [21]:
# Retorna a precisão do modelo
accuracy_score(ytrue, yhat)

0.8

In [33]:
cap.release()
cv2.destroyAllWindows()

In [25]:
sequence = []      # Guarda os frames
sentence = []      # Guarda quais foram as previsões feitas pelo modelo para formar frases 
threshold = 0.7    # Mínimo para renderizar os resultados, evitando resultados de baixa confiança

#Acessa a webcam - valor (0) representa o hardware
cap = cv2.VideoCapture(0)

#Define o modelo utilizado
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #Seleciona o frame atual da webcam
        ret, frame = cap.read()

        #Processa previsoes
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        #Desenha os pontos
        draw_landmarks(image, results)
        
        #Previsoes
        #Pega os pontos que estao aparecendo na camera e inserem em um array
        #Cada sequencia é feita com os 30 ultimos frames que depois será passada para o modelo
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-60:]
        
        if len(sequence) == 60:
            #expand_dims permite passar uma sequencia por vez
            res = model.predict(np.expand_dims(sequence[-30], axis=0))[0]
            #print(actions[np.argmax(res)])
            
            #Visualização
            #Compara se a probabilidade de certo sinal é maior que o minimo para mostrar
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    # Verifica se o sinal já não está incluido na frase
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            #Caso a frase fique muito grande, guarda apenas os ultimos valores
            if len(sentence) > 5:
                sentence = sentence[-5:]
        
        #Renderiza predições e a frase
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        #Faz o display do frame
        if ret == True: 
            cv2.imshow('Webcam', image)

            #Encerra a webcam 
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

ValueError: in user code:

    File "C:\Users\FN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\FN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\FN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\FN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\FN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\FN\AppData\Loc